# Optimex Example: Hydrogen Pathway with Multi-level Choices

This notebook extends the explicit-products example and builds a **two-level decision problem** set in energy & process engineering:

- **Level 1 (Hydrogen technology choice):** Steam‑methane reforming (SMR) vs. PEM electrolysis.
- **Level 2 (Power supply choice for PEM):** Grid power vs. a dedicated wind PPA.

Demand for hydrogen grows from 2025–2035. The background system (grid mix and gas supply) evolves in 2035, so we expect **switches over time**: SMR dominates early years, while cleaner grids and a wind PPA make electrolysis preferable later. Values are illustrative but within realistic order of magnitude for process emissions and leakage.

In [ ]:

from datetime import datetime
import numpy as np
import bw2data as bd
from bw_temporalis import TemporalDistribution

bd.projects.set_current("optimex_energy_switches")


In [ ]:

# --- temporary patch: matrix_utils uses `.A1` which sparse matrices lack in SciPy ---
import matrix_utils.array_mapper as _am
import numpy as _np

def _safe_map_array(self, array: _np.ndarray) -> _np.ndarray:
    self._check_input_array(array)
    if array.shape == (0,):
        return array.copy()
    if getattr(self, "empty_input", False):
        if self.empty_ok:
            return _np.zeros_like(array) - 1
        else:
            raise _am.EmptyArray("Can't map with empty input array")
    result = _np.zeros_like(array) - 1
    mask = array <= self.max_value
    if mask.any():
        # use dense array conversion instead of `.A1` to support SciPy sparse matrices
        result[mask] = self.matrix[array[mask], _np.zeros_like(array[mask])].toarray().ravel() - 1
    return result

_am.ArrayMapper.map_array = _safe_map_array


In [ ]:

# BIOSPHERE
biosphere_data = {
    ("biosphere3", "CO2"): {"type": "emission", "name": "carbon dioxide"},
    ("biosphere3", "CH4"): {"type": "emission", "name": "methane, fossil"},
    ("biosphere3", "NOx"): {"type": "emission", "name": "nitrogen oxides"},
}
bd.Database("biosphere3").write(biosphere_data)


In [ ]:

# CHARACTERIZATION METHODS (illustrative factors)
bd.Method(("GWP", "example")).write([
    (("biosphere3", "CO2"), 1.0),
    (("biosphere3", "CH4"), 28.0),
])

bd.Method(("NOx-health", "example")).write([
    (("biosphere3", "NOx"), 0.5),
])


In [ ]:

# BACKGROUND 2025 (fossil-heavy grid, higher gas leakage)
db_2025_data = {
    ("db_2025", "grid_power"): {
        "name": "2025 grid electricity",
        "reference product": "electricity, medium voltage",
        "unit": "MWh",
        "exchanges": [
            {"amount": 1, "type": "production", "input": ("db_2025", "grid_power")},
            {"amount": 0.45, "type": "biosphere", "input": ("biosphere3", "CO2")},
            {"amount": 0.0003, "type": "biosphere", "input": ("biosphere3", "CH4")},
        ],
    },
    ("db_2025", "pipeline_gas"): {
        "name": "2025 pipeline natural gas",
        "reference product": "natural gas, high pressure",
        "unit": "MWh_higher_heating",
        "exchanges": [
            {"amount": 1, "type": "production", "input": ("db_2025", "pipeline_gas")},
            {"amount": 0.06, "type": "biosphere", "input": ("biosphere3", "CH4")},
            {"amount": 0.02, "type": "biosphere", "input": ("biosphere3", "CO2")},
        ],
    },
}
bg_2025 = bd.Database("db_2025")
bg_2025.write(db_2025_data)
bg_2025.metadata["representative_time"] = datetime(2025, 1, 1).isoformat()
bg_2025.register()

# BACKGROUND 2035 (cleaner grid, lower leakage)
db_2035_data = {
    ("db_2035", "grid_power"): {
        "name": "2035 grid electricity",
        "reference product": "electricity, medium voltage",
        "unit": "MWh",
        "exchanges": [
            {"amount": 1, "type": "production", "input": ("db_2035", "grid_power")},
            {"amount": 0.12, "type": "biosphere", "input": ("biosphere3", "CO2")},
            {"amount": 0.0001, "type": "biosphere", "input": ("biosphere3", "CH4")},
        ],
    },
    ("db_2035", "pipeline_gas"): {
        "name": "2035 pipeline natural gas with leak reduction",
        "reference product": "natural gas, high pressure",
        "unit": "MWh_higher_heating",
        "exchanges": [
            {"amount": 1, "type": "production", "input": ("db_2035", "pipeline_gas")},
            {"amount": 0.025, "type": "biosphere", "input": ("biosphere3", "CH4")},
            {"amount": 0.01, "type": "biosphere", "input": ("biosphere3", "CO2")},
        ],
    },
    ("db_2035", "wind_power"): {
        "name": "2035 wind PPA",
        "reference product": "electricity, wind",
        "unit": "MWh",
        "exchanges": [
            {"amount": 1, "type": "production", "input": ("db_2035", "wind_power")},
            {"amount": 0.02, "type": "biosphere", "input": ("biosphere3", "CO2")},
        ],
    },
}
bg_2035 = bd.Database("db_2035")
bg_2035.write(db_2035_data)
bg_2035.metadata["representative_time"] = datetime(2035, 1, 1).isoformat()
bg_2035.register()


In [ ]:

# FOREGROUND WITH TWO DECISION LEVELS

time_steps = np.array(range(11), dtype="timedelta64[Y]")  # 2025-2035 inclusive

foreground_data = {
    # Products (functional and intermediate)
    ("foreground", "Hydrogen"): {
        "name": "Hydrogen, gaseous",
        "unit": "kg",
        "type": bd.labels.product_node_default,
    },
    ("foreground", "PEM power"): {
        "name": "Electricity for PEM stack",
        "unit": "MWh",
        "type": bd.labels.product_node_default,
    },

    # Level 1 choice: SMR route
    ("foreground", "H2_SMR"): {
        "name": "Hydrogen via SMR",
        "location": "Gulf Coast",
        "type": bd.labels.process_node_default,
        "operation_time_limits": (0, 7),  # 2025-2032
        "exchanges": [
            {
                "amount": 1,
                "type": bd.labels.production_edge_default,
                "input": ("foreground", "Hydrogen"),
                "temporal_distribution": TemporalDistribution(
                    date=time_steps,
                    amount=np.array([0.6,0.6,0.6,0.55,0.5,0.4,0.3,0.2,0.1,0.05,0])
                ),
                "operation": True,
            },
            {
                "amount": 0.18,  # MWh gas / kg H2 (HHV ~50 kWh/kg; SMR ~65% eff.)
                "type": bd.labels.consumption_edge_default,
                "input": ("db_2025", "pipeline_gas"),
                "temporal_distribution": TemporalDistribution(
                    date=time_steps,
                    amount=np.array([1,1,1,0.8,0.6,0.4,0.3,0.2,0.1,0,0])
                ),
                "operation": True,
            },
            {
                "amount": 9.0,  # process CO2 kg/kg H2 after minor CCS
                "type": bd.labels.biosphere_edge_default,
                "input": ("biosphere3", "CO2"),
                "temporal_distribution": TemporalDistribution(date=time_steps, amount=np.array([0.6,0.6,0.6,0.55,0.5,0.4,0.3,0.2,0.1,0.05,0])),
                "operation": True,
            },
            {
                "amount": 0.01,
                "type": bd.labels.biosphere_edge_default,
                "input": ("biosphere3", "NOx"),
                "temporal_distribution": TemporalDistribution(date=time_steps, amount=np.array([0.6,0.6,0.6,0.55,0.5,0.4,0.3,0.2,0.1,0.05,0])),
                "operation": True,
            },
        ],
    },

    # Level 1 choice: PEM route (consumes intermediate electricity)
    ("foreground", "H2_PEM"): {
        "name": "Hydrogen via PEM electrolysis",
        "location": "Texas wind corridor",
        "type": bd.labels.process_node_default,
        "operation_time_limits": (2, 10),  # 2027-2035
        "exchanges": [
            {
                "amount": 1,
                "type": bd.labels.production_edge_default,
                "input": ("foreground", "Hydrogen"),
                "temporal_distribution": TemporalDistribution(
                    date=time_steps,
                    amount=np.array([0,0,0.15,0.25,0.35,0.5,0.65,0.8,0.9,1.0,1.0])
                ),
                "operation": True,
            },
            {
                "amount": 0.055,  # MWh electricity per kg H2 (55 kWh/kg)
                "type": bd.labels.consumption_edge_default,
                "input": ("foreground", "PEM power"),
                "temporal_distribution": TemporalDistribution(
                    date=time_steps,
                    amount=np.array([0,0,0.15,0.25,0.35,0.5,0.65,0.8,0.9,1.0,1.0])
                ),
                "operation": True,
            },
            {
                "amount": 0.2,  # small balance-of-plant losses
                "type": bd.labels.biosphere_edge_default,
                "input": ("biosphere3", "CO2"),
                "temporal_distribution": TemporalDistribution(date=time_steps, amount=np.array([0,0,0.15,0.25,0.35,0.5,0.65,0.8,0.9,1.0,1.0])),
                "operation": True,
            },
        ],
    },

    # Level 2 choice: electricity supply for PEM
    ("foreground", "Grid_supply"): {
        "name": "Grid supply for PEM",
        "type": bd.labels.process_node_default,
        "operation_time_limits": (0, 10),
        "exchanges": [
            {
                "amount": 1,
                "type": bd.labels.production_edge_default,
                "input": ("foreground", "PEM power"),
                "temporal_distribution": TemporalDistribution(date=time_steps, amount=np.ones(11)),
                "operation": True,
            },
            {
                "amount": 1,
                "type": bd.labels.consumption_edge_default,
                "input": ("db_2025", "grid_power"),
                "temporal_distribution": TemporalDistribution(
                    date=time_steps,
                    amount=np.array([1,1,1,1,0.8,0.6,0.4,0.2,0,0,0])
                ),
                "operation": True,
            },
            {
                "amount": 1,
                "type": bd.labels.consumption_edge_default,
                "input": ("db_2035", "grid_power"),
                "temporal_distribution": TemporalDistribution(
                    date=time_steps,
                    amount=np.array([0,0,0,0,0.2,0.4,0.6,0.8,1,1,1])
                ),
                "operation": True,
            },
        ],
    },
    ("foreground", "Wind_PPA"): {
        "name": "Dedicated wind PPA",
        "type": bd.labels.process_node_default,
        "operation_time_limits": (5, 10),  # available from 2030
        "exchanges": [
            {
                "amount": 1,
                "type": bd.labels.production_edge_default,
                "input": ("foreground", "PEM power"),
                "temporal_distribution": TemporalDistribution(
                    date=time_steps,
                    amount=np.array([0,0,0,0,0,0.2,0.5,0.8,1,1,1])
                ),
                "operation": True,
            },
            {
                "amount": 1,
                "type": bd.labels.consumption_edge_default,
                "input": ("db_2035", "wind_power"),
                "temporal_distribution": TemporalDistribution(
                    date=time_steps,
                    amount=np.array([0,0,0,0,0,0.2,0.5,0.8,1,1,1])
                ),
                "operation": True,
            },
        ],
    },
}

fg = bd.Database("foreground")
fg.write(foreground_data)
fg.register()


### Defining Hydrogen Demand (2025–2035)

In [ ]:

# Temporal demand: modest early demand, rapid scale-up after 2030
years = list(range(2025, 2036))
td_demand = TemporalDistribution(
    date=np.array([datetime(y, 1, 1).isoformat() for y in years], dtype='datetime64[s]'),
    amount=np.array([5, 6, 8, 10, 12, 15, 30, 40, 45, 50, 55]),
)
functional_demand = {bd.get_node(database="foreground", name="Hydrogen"): td_demand}


## optimex configuration

In [ ]:

from optimex import lca_processor

lca_config = lca_processor.LCAConfig(
    demand=functional_demand,
    temporal={
        "start_date": datetime(2025, 1, 1),
        "temporal_resolution": "year",
        "time_horizon": 20,
    },
    characterization_methods=[
        {"category_name": "climate_change", "brightway_method": ("GWP", "example"), "metric": "CRF"},
        {"category_name": "air_pollution", "brightway_method": ("NOx-health", "example")},
    ],
)



With the foreground, background, and temporal demand in place, we can gather the life-cycle inventory and prepare it for optimization.
Key moving parts:
- **Level 1 choice:** SMR vs. PEM for hydrogen.
- **Level 2 choice:** Grid vs. wind PPA to supply PEM electricity.
- **Background evolution:** Cleaner 2035 grid and lower gas leakage.
- **Temporal demand ramp:** Drives a likely switch from SMR in the 2020s to PEM+wind in the 2030s.


In [ ]:
from optimex import lca_processor
lca_data_processor = lca_processor.LCADataProcessor(lca_config)


In [ ]:
from optimex import converter
manager = converter.ModelInputManager()
optimization_model_inputs = manager.parse_from_lca_processor(lca_data_processor)


In [ ]:
# optimization_model_inputs.model_dump()  # uncomment to inspect the assembled data


### Creating and Solving the Optimization Model

In [ ]:

from optimex import optimizer

model = optimizer.create_model(
    optimization_model_inputs,
    name="hydrogen_multilevel_switch",
    objective_category="climate_change",
    flexible_operation=True,
)


In [ ]:
m, obj, results = optimizer.solve_model(model, solver_name="glpk", tee=False)
obj


## Postprocessing

In [ ]:

import pyomo.environ as pyo
operation_matrix = {(t, p): pyo.value(m.var_operation[p, t]) for p in m.PROCESS for t in m.SYSTEM_TIME}
operation_matrix


In [ ]:

from optimex import postprocessing
pp = postprocessing.PostProcessor(m)

# Impact and operation visuals
pp.plot_impacts()
pp.plot_installation()
pp.plot_production_and_demand()
